# 2.收缩方法(shrink method)
 又称正则化（regularization)

## 2.1 背景简介：
如果数据的特征比样本点要多，即($n<p$)时，输入数据的矩阵不是满秩矩阵。就不能再使用之前的线性回归方法对参数进行求解。$X^TX$矩阵奇异，逆矩阵不存在。为了解决这一问题，统计学家引入了一些缩减方法，如：岭回归，Lasso，LAR，PCA回归以及子集选择等。本节将逐一介绍。  

## 2.2 岭回归(Ridge regression)
岭回归是一种专用于共线性数据分析的有偏估计回归方法，实际上是一种改良的最小二乘估计法，通过放弃最小二乘法的无偏性，以损失部分信息，降低精度为代价获得回归系数更符合实际，更可靠的回归方法，对病态数据的拟合要强于最小二乘法。
### 思想：  
简单说来，岭回归就说在矩阵$X^TX$上加一个$\lambda I$从而使得矩阵非奇异，进而能对$X^TX+\lambda I$求逆，其中矩阵$I$是一个$p\times p$维的单位矩阵，对角线上元素全为1，其他元素全为0.而$\lambda$是一个用户定义的数值。  
在这种情况下，回归系数的计算公式将变成：  
$$\hat{\theta}=(X^TX+\lambda I)^{-1}X^Ty\qquad (*)$$


### 理论推导：
岭回归通过对系数大小施加惩罚，在平方误差的基础上添加了正则项，解决了普通最小二乘法的一些问题。岭系数最小化惩罚残差平方和：  
$$ \min\limits_{\theta}\dfrac{1}{2}\lVert X\theta-y \rVert _2^2+\dfrac{1}{2}\lambda \lVert \theta \rVert _2^2\qquad (**)$$
这里，$\lambda\ge0$是控制收缩量的复杂参数:$\lambda$值越大，收缩量越大，从而系数对于共线性更加稳健。

回归系数的计算公式也可由损失函数最小化求得。数学推导如下：
$$\dfrac{1}{2}\lVert X\theta-y \rVert _2^2+\dfrac{1}{2}\lambda \lVert \theta \rVert _2^2=
\dfrac{1}{2}\theta^TX^TX\theta-\theta^TX^Ty+\dfrac{1}{2}y^Ty+\dfrac{\lambda}{2}\theta^T\theta$$
对函数求导，得$X^TX\theta-X^Ty+\lambda\theta$，令其等于0，得到参数的估计值与(*)式结果相同。
### 表达岭回归的一个等价方法是：
$$\hat{\theta}=arg\min\limits_{\theta}\lVert X\theta-y \rVert _2^2$$
$$受限于 \lVert \theta \rVert _2^2\le s$$  

在参数$\lambda$和上式的s之间存在一个一一对应。当线性回归模型中存在多个相关变量时，它们的系数确定性变差，并呈现高方差。在一个变量上的很大的正系数可能被在其变量上类似大小的负系数抵消。通过在系数上施加一个量约束s,可以避免这种现象发生。  
在输入缩放s时，岭解是不等价的，因此在求解(**)式之前，通常要对输入标准化。

#### 思考：岭回归中的岭指的是什么？  
岭回归使用了单位矩阵乘以常量$\lambda$，观察其中的单位矩阵，可以看到1贯穿整个对角线，其余元素全为0.形象地，在0构成的平面上有一条1组成的'岭'，这就是岭的由来。
#### 注意：
通常岭回归方程的R平方值会稍低于普通回归分析，但回归系数的显著性往往明显高于普通回归，在存在共线性问题和病态数据偏多的研究中有较大的实用价值。
### 问题：岭回归要对数据标准化吗？
为了使用岭回归和缩减技术，需要对特征做标准化处理。具体的做法是所有的特征都减去各自的均值并除以方差。  因为无论ridge regression和lasso，它们的损失函数求解，不中心化求出来的$\theta$都不一样，普通的线性回归就没有任何问题。  
注意截距$\theta_0$是排除在罚项之外的。截距的罚将使得过程依赖于Y的原点选择。  
在本节中，假定已进行中心化，从而输入矩阵X有p列(而不是p+1列)。

In [45]:
from sklearn import preprocessing
import numpy as np
def ridgereg(X_stand,y,lam):
    p=X_stand.shape[1]
    xTxlam=X_stand.T*X_stand+lam*np.eye(p)
    if np.linalg.det(xTxlam)== 0:
        print('This matrix is singular')
        return
    theta=xTxlam.I*(X_stand.T*y)
    return theta
def ridgetest(X,y):
    n=len(y)
    Xmat=np.hstack((np.ones((n,1)),np.mat(X)))
    ymat=np.mat(y).T
    Xmean=np.mean(Xmat,axis=0)
    Xvar=np.var(Xmat,axis=0)
    Xmat=np.mat(preprocessing.scale(Xmat))
    # 使用不同的lambda，看一下效果
    numTest=10
    Wmat=np.zeros((numTest,Xmat.shape[1]))
    for i in range(numTest):
        ws=ridgereg(Xmat,ymat,np.exp(i-10))
        Wmat[i,:]=ws.T
    return Wmat
# 为什么要检查行列式是否为0:
# 当lam设定为0时，岭回归回到普通线性回归，有可能会产生错误。

In [46]:
X=[[0,0],[1,1],[2,2]]
y=[0,1,2]
w=ridgetest(X,y)
print('w is :\n',w)

w is :
 [[ 0.          0.4082452   0.4082452 ]
 [ 0.          0.40823989  0.40823989]
 [ 0.          0.40822547  0.40822547]
 [ 0.          0.40818625  0.40818625]
 [ 0.          0.4080797   0.4080797 ]
 [ 0.          0.40779035  0.40779035]
 [ 0.          0.40700586  0.40700586]
 [ 0.          0.40488859  0.40488859]
 [ 0.          0.39924301  0.39924301]
 [ 0.          0.38466334  0.38466334]]


## 2.3 Lasso
Lasso(Least absolute shrinkage and selection operator)

Lasso 是一种估计稀疏线性模型的方法.由于它倾向具有少量参数值的情况，对于给定解决方案是相关情况下，有效的减少了变量数量。 因此，Lasso及其变种是压缩感知(压缩采样)的基础。

## 2.4 弹性网

## 2.5 SCAD

## 2.6 group lasso